In [1]:

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

# load train and test dataset
def load_dataset():
    # load dataset
    (trainX, trainY), (testX, testY) = mnist.load_data()
    # reshape dataset to have a single channel
    trainX = trainX.reshape((trainX.shape[0], 400, 400, 1))
    testX = testX.reshape((testX.shape[0], 400, 400, 1))
    # one hot encode target values
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

# define cnn model
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(400, 400, 1)))
    model.add(MaxPooling2D((2, 2))) # important feature extraction, noice reduction
    # https://keras.io/api/layers/pooling_layers/max_pooling2d/
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')) # convolutional layer
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform')) 
    model.add(MaxPooling2D((2, 2))) # 
    model.add(Flatten()) # convert into a one vector
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform')) # output as 100 values
    model.add(Dense(10, activation='softmax')) # output as 10 classes
    # compile model
    opt = SGD(lr=0.01, momentum=0.9) # adding optimizer, Gradie
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
    # save model
    model.save('final_model.h5')

# entry point, run the test harness
run_test_harness()

ModuleNotFoundError: No module named 'keras'